# 유사한 단어 찾기 게임`

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
2. 워드 임베딩 모델을 학습시킨다.
3. 단어 A,B,C를 입력받는다.
4. A:B = C: answer 관계에 대응하는 answer를 찾는 게임을 만든다.
    - ex) A: 산, B: 바다, C:나무 / D -> 물

In [ ]:
import json

# JSON 파일 읽기
with open('./practice_data/korquad2.1_dev_00.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 보기 좋게 정리된 JSON 파일 쓰기
with open('formatted_file.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [53]:
import json
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from konlpy.tag import Okt # 형태소 분석기
import gensim
from gensim.models import Word2Vec

In [ ]:
with open('./practice_data/formatted_file.json','r',encoding='utf-8') as file:
    data = json.load(file)

# title, html_answer_text, question만 추출하여 DataFrame 생성
title = data['data'][0]['title']
qas = data['data'][0]['qas']

rows = []
for qa in qas:
    question = qa['question']
    html_answer_text = qa['answer']['html_answer_text']
    rows.append({"title": title, "question": question, "html_answer_text": html_answer_text})

df = pd.DataFrame(rows)
print(df)

In [ ]:
with open('./practice_data/formatted_file.json','r',encoding='utf-8') as file:
    data = json.load(file)

print(json.dumps(data['data'][0], indent='\t', ensure_ascii=False))

In [ ]:
with open('./practice_data/formatted_file.json','r',encoding='utf-8') as file:
    data = json.load(file)

print(json.dumps(data['data'][0], indent='\t', ensure_ascii=False))

{
	"title": "2016년_하계_올림픽_배구_남자_선수_명단",
	"url": "https://ko.wikipedia.org/wiki/2016년_하계_올림픽_배구_남자_선수_명단",
	"context": "<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<title>2016년 하계 올림픽 배구 남자 선수 명단 - 위키백과, 우리 모두의 백과사전</title>\n\n\n<link>\n\n<meta>\n<link>\n<meta>\n<meta>\n<meta>\n<meta>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n\n</head>\n<body>\n<div></div>\n<div></div>\n<div>\n<a></a>\n<div></div>\n<div>\n</div>\n<h1>2016년 하계 올림픽 배구 남자 선수 명단</h1>\n<div>\n<div>위키백과, 우리 모두의 백과사전.</div>\n<div></div>\n<div></div>\n<a>둘러보기로 가기</a>\n<a>검색하러 가기</a>\n<div><div><table>\n<tbody><tr>\n<th colspan=\"5\"><a>2016년 하계<br/>올림픽 배구</a>\n</th></tr>\n<tr>\n<th colspan=\"5\"><a><img></a>\n</th></tr>\n<tr>\n<td>경기\n</td>\n<td> \n</td>\n<td><a>남자</a>\n</td>\n<td> \n</td>\n<td><a>여자</a>\n</td></tr>\n<tr>\n<td>선수명단\n</td>\n<td> \n</td>\n<td><a>남자</a>\n</td>\n<td> \n</td>\n<td><a>여자</a>\n</td></tr>\n<tr>\n<th colspan=\"5\">비치발리볼<br/><a><img></a>\n</th></tr>\n<tr>\n<td>

In [ ]:
import json
import pandas as pd

with open('./practice_data/formatted_file.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

title = data['data'][0]['title']
qas = data['data'][0]['qas']

data_list = []
for qa in qas:
    question = qa['question']
    html_answer_text = qa['answer']['html_answer_text']
    data_list.append({'title': title, 'question': question, 'html_answer_text': html_answer_text})

df = pd.DataFrame(data_list)

df

,title,question,html_answer_text
0,2016년_하계_올림픽_배구_남자_선수_명단,파스칼레 소틸레의 스파이크 높이는 몇 cm인가?,<td>332cm</td>
1,2016년_하계_올림픽_배구_남자_선수_명단,"2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 201...",MKS 벵진


In [48]:
import json
import pandas as pd

# JSON 파일 읽기
with open('./practice_data/formatted_file.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 모든 데이터를 순회하며 필요한 정보 추출
data_list = []
for item in data['data']:
    title = item['title']
    qas = item['qas']
    for qa in qas:
        question = qa['question']
        html_answer_text = qa['answer']['html_answer_text']
        data_list.append({'title': title, 'question': question, 'html_answer_text': html_answer_text})

# DataFrame으로 변환
df = pd.DataFrame(data_list)

# DataFrame 출력
df

,title,question,html_answer_text
0,2016년_하계_올림픽_배구_남자_선수_명단,파스칼레 소틸레의 스파이크 높이는 몇 cm인가?,<td>332cm</td>
1,2016년_하계_올림픽_배구_남자_선수_명단,"2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 201...",MKS 벵진
2,혈액_검사,혈액 검사에서 나트륨의 정상범위는 최소 136mmol/L에서 최대 몇 mmol/L까...,<td>145</td>
3,안토닌_드보르자크,안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는?,사이프러스
4,조석_가속,에드먼드 핼리가 달의 운동이 빨라지고 있다고 주장한 년도는 언제일까?,1695년
...,...,...,...
1720,토마스_헤슬러,1997-98시즌 마지막 3개의 경기에서 토마스 헤슬러는 몇 개의 골을 기록하였는가?,4골
1721,토마스_헤슬러,독일의 축구선수 토마스 헤슬러는 국가대표로서 어떤 활약을 했을까?,"<table class=""wikitable"" style=""text-align: ce..."
1722,토마스_헤슬러,토마스 헤슬러는 선수시절 리그에서 총 몇 경기에 출전했는가?,539
1723,나노블럭,나노블럭의 정품을 생산하는 기업은 어디인가?,"<a class=""mw-disambig"" href=""/wiki/%EC%B9%B4%E..."


In [49]:
# 한글이 아닌 데이터 제거
df['question'] =df['question'].replace(r'[^0-9가-힣ㄱ-ㅎㅏ-ㅣ]', '', regex=True) # 한글이 아닌 데이터 제거, 자음 혹은 모음으로만 이뤄진 단어들은 남겨둠

In [50]:
# 전처리
from tqdm import tqdm
okt = Okt()
ko_stopwards = ['은','는','이','가','을','를','와','과','들','도','부터','까지','에','나','너','그','걔','얘']

preprocessed_data = []

for sentence in tqdm(df['question']): # tqdm : 진행상황을 시각화 해주는 라이브러리
    tokens = okt.morphs(sentence, stem=True)    #morphs : 형태소 분석기, stem : 어간 추출
    tokens = [token for token in tokens if token not in ko_stopwards]
    preprocessed_data.append(tokens)
    
preprocessed_data[:5]

100%|██████████| 1725/1725 [00:15<00:00, 112.03it/s]


[['파스', '칼레', '소', '틸레', '의', '스파이크', '높이다', '몇', '인가'],
 ['2016년', '하계올림픽캐나다남자배구대표팀선수들중타일러샌더스는', '201516년', '에어느팀소속이었나'],
 ['혈액',
  '검사',
  '에서',
  '나트륨',
  '의',
  '정상',
  '범위',
  '최소',
  '136',
  '에서',
  '최대',
  '몇',
  '이다'],
 ['안토닌', '드보르자크', '의', '작품', '중', '18', '개', '의', '가곡', '으로', '이루어지다', '노래'],
 ['에드먼드', '핼리', '달', '의', '운동', '빨르다', '있다', '주장', '한', '년도', '언', '제일', '끄다']]

In [51]:
model = Word2Vec(
    sentences=preprocessed_data,
    vector_size=100,
    window=5,
    min_count=5,
    sg=0 # CBOW
)

model.wv.vectors.shape

(436, 100)

In [52]:
model.wv.save_word2vec_format('./data/practice_model_w2v')

In [58]:
import random
# 임베딩 모델 로드
from gensim.models import KeyedVectors

load_model = KeyedVectors.load_word2vec_format('./data/practice_model_w2v')

def find_analogy(model, word_a, word_b, word_c):
    """
    A:B = C:D 관계에서 D를 찾는 함수
    """
    try:
        result = model.most_similar(positive=[word_c, word_b], negative=[word_a], topn=1)
        return result[0][0]
    except KeyError as e:
        return str(e)

def get_similar_words(model, threshold=0.8):
    """
    유사도가 threshold 이상인 단어 쌍을 반환하는 함수
    """
    similar_pairs = []
    vocab = list(model.index_to_key)
    for word in vocab:
        similar_words = model.most_similar(word, topn=10)
        for similar_word, similarity in similar_words:
            if similarity >= threshold:
                similar_pairs.append((word, similar_word))
    return similar_pairs

def play_game(model):
    print("유사한 단어 찾기 게임을 시작합니다!")
    print("A:B = C:D 관계에서 D를 찾으세요.")
    
    similar_pairs = get_similar_words(model)
    
    while True:
        word_a, word_b = random.choice(similar_pairs)
        word_c = random.choice(model.index_to_key)
        
        print(f"A: {word_a} , B: {word_b} , C: {word_c} ")
        word_d = input("D를 입력하세요 (종료하려면 'exit' 입력): ")
        if word_d == 'exit':
            break
        
        answer = find_analogy(model, word_a, word_b, word_c)
        print(f"{word_a}:{word_b} = {word_c}:{answer}")
        print()

# 게임 시작
play_game(load_model)

유사한 단어 찾기 게임을 시작합니다!
A:B = C:D 관계에서 D를 찾으세요.
A: 역할 , B: 되다 , C: 특징 
역할:되다 = 특징:가다

A: 으로 , B: 중 , C: 가수 
으로:중 = 가수:의

A: 평가 , B: 되다 , C: 그룹 
평가:되다 = 그룹:일

A: 이르다 , B: 몇 , C: 방법 
